# Машинное обучение

In [1]:
import numpy as np
import pandas as pd

## Предобработка данных

### Работа с пропусками с категориальными данными

- заменить на "неизвестно"
- заменить наиболее популярным значением
- дропнуть

### Работа с пропусками с числовыми данными

- заменить на среднее значение
- `ffill`/`bfill`/`interpolate`, где это имеет смысл (например, во временных рядах)
- дропнуть


### Нормализация числовых признаков

- привести к диапазону от 0 до 1 по формуле

$x_{new} = \frac{x - x_{min}}{x_{max} - x_{min}}$

- вычесть среднее и разделить на среднеквадратичное отклонение

$x_{new} = \frac{x - x_{mean}}{\sigma}$

### Обработка категориальных признаков

- OneHotEncoding - заменяем колонку с $n$ значениями на $n$ колонок, в каждой из которых стоит 0 или 1, в зависимости от того, принадлежит ли объект этой категории. Универсальный способ, но значительно увеличивает размерность в данных (нужно много данных, чтобы модель хорошо обучилась).
- Закодировать категорию от 1 до n. Хорошо сработает, если категории можно упорядочить (например, год выпуска машины, грейд работника) или если всего две категории (например, пол) и нет смысла добавлять вторую колонку как в one hot encoding.
- Не используем категориальный признак


## Задача

Для данных о погоде создать категорию "время года" и закодировать ее при помощи OneHotEncoder. Проще всего применить OneHotEncoder при помощи `pandas.get_dummies()`

Преобразовать данные о температуре двумя способами.

In [54]:
# Пример

series = pd.Series(['blue', 'red', 'blue', 'red', 'green'])
pd.get_dummies(series)

,blue,green,red
0,1,0,0
1,0,0,1
2,1,0,0
3,0,0,1
4,0,1,0


In [55]:
# Ваш код здесь


## Модель случайного леса

Обучим 100 деревьев решений на разных подмножествах обучающей выборки (выбирая для каждого дерева случайные строки и столбцы из $X$). 

Ответ будем получать голосованием.

Например, за то, что объект принадлежит классу 1 проголосовало 90 деревьев, а за то, что объект принадлежит классу 2, проголосовало 10 деревьев. Итого - берем ответ 1.

Такая модель называется моделью случайного леса

Основные аргументы:

- `n_estimators` - максимальное число деревьев в лесу
- `criterion` - критерий для создания узла: или критерий Джини gini (по умолчанию), или энтропия entropy.
- `max_depth` - максимальная глубина дерева
- `max_features` - максимальное число атрибутов, которые будут проверены при создании узла, по умолчанию это равно корню квадратному из числа всех атрибутов в данных.
- `max_samples` - максимальное число примеров используемых для одного дерева (примеры выбираются случайно).


Сам объект RandomForestClassifier имеет поля:

- `estimators_` - список объектов деревьев (типа DecisionTreeClassifier) в этом лесу
- `classes_` - метки классов
- `n_classes_` - число классов
- `n_features_` - число атрибутов
- `n_outputs_` - число выходов
- `feature_importances_` - оценка важности атрибутов. Очень полезные сведения, показывающие какой вклад дает тот или иной атрибут в точность решения задачи. Полезно, например, для существенного уменьшения размера дерева: удалив атрибуты с малой важностью не сильно потеряем в точности. Полезно и для интерпретируемости результатов. Вычисляется как суммарная величина уменьшения критерия неопределенности для этого атрибута.

In [10]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import load_digits
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split

In [9]:
data = load_digits()
x = data.data
y = data.target

In [11]:
x_train, x_test, y_train, y_test = train_test_split(x, y)

In [12]:
model = RandomForestClassifier()
model.fit(x_train, y_train)

y_pred = model.predict(x_test)
accuracy_score(y_test, y_pred)

0.9711111111111111

In [13]:
confusion_matrix(y_test, y_pred)

array([[47,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0, 40,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 1,  0, 40,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0, 49,  0,  1,  0,  0,  2,  0],
       [ 0,  0,  0,  0, 45,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0, 42,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  1,  0, 45,  0,  1,  0],
       [ 0,  0,  0,  0,  1,  0,  0, 42,  0,  0],
       [ 0,  2,  0,  0,  0,  1,  0,  0, 44,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  3,  0, 43]], dtype=int64)

## Задача

Перебрать параметры `n_estimators`, `criterion` и `max_depth` и найти оптимальные. 

Для решения можно написать тройной цикл, в каждом из которых можно перебирать значения параметров.

In [17]:
def get_accuracy_of_model(x_train, y_train, x_test, y_test, n_estimators, criterion, max_depth):
    """Функция принимает на вход треин и тест выборки и параметры модели. На выходе дает accuracy."""
    model = RandomForestClassifier(n_estimators=n_estimators, criterion=criterion, max_depth=max_depth)
    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)
    return accuracy_score(y_test, y_pred)

In [1]:
n_estimators = [5, 100, 300, 500]
criterions = ['gini', 'entropy']
max_depths = [3, 5, 7, 10]

# Ваш код здесь


## Подбор гиперпараметров модели

Иногда нужно перебирать множество параметров и не хочется писать много циклов. Для таких случаев в sklearn реализован `GridSearchCV` и `RandomSearchCV`.

In [23]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

In [35]:
params = {
    'n_estimators': [5, 100, 500, 1000],
    'criterion': ['gini', 'entropy'],
    'max_depth': [3, 5, 7, 10]
}

In [36]:
%%time
search = GridSearchCV(RandomForestClassifier(), params)
search.fit(x_train, y_train)
search.best_estimator_

Wall time: 2min 54s


RandomForestClassifier(max_depth=10, n_estimators=1000)

In [37]:
%%time
r_search = RandomizedSearchCV(RandomForestClassifier(), params)
r_search.fit(x_train, y_train)
r_search.best_estimator_

Wall time: 1min 2s


RandomForestClassifier(max_depth=10, n_estimators=1000)

## Задача

Построить модель SGDClassifier и осуществить подбор гиперпараметров при помощи случайного поиска (список гиперпараметров можно посмотреть в документации).

In [45]:
from sklearn.linear_model import SGDClassifier

In [ ]:
# Ваш код здесь


## Ансамблевые модели

Объединяя множество простых моделей в одну сложную, можно получить результат гораздо лучше.

Есть несколько способов создания ансамблей:

- **Бэггинг** (Bagging, сокращение от Bootstrap Aggregation) - однородные модели (из одного класса) обучаются независимо друг от друга на разных случайных подвыборках и разных признаках. Итоговый прогноз считается при помощи голосования моделей. Пример: случайный лес
- **Бустинг** - однородные модели строятся последовательно, каждая следующая модель старается минимизировать ошибку предыдущей. Пример: градиентный бустинг
- **Стекинг** - разнородные модели обучаются на всех данных (или на подвыборках) и поверх них строится модель, которая минимизирует общую ошибку.

На основе стекинга можно строить очень сложные модели, которые будут долго обучаться и иметь очень высокую точность.

## Градиентный бустинг

Идея бустинга заключается в следующем:

* Пусть мы имеем входы $X$ и указания учителя $Y$.
* Обучим первую модель на этих данных, если модель простая,  то обучение будет не очень точным и выход модели будет $F1$, а должны были получить $Y$
* Обучим вторую модель на тех же данных, но для нее указания учителя сделаем $Y-F1$, получим выход $F2$, который тоже может отличаться от желаемого. 
* тогда обучим третью модель, для которой указания учителя будут $Y-F2$, получим выход $F3$.
* будем продолжать обучать все новые и новые модели, пока не надоест или пока не достигнем хорошего обучения или других критериев останова. 
* Результат ансамбля, выход $F=F1+F2+F3+...$

Если внимательно присмотреться, то величина $(Y - F1)$ это градиент по $F1$ (со знаком минус) от средневквадратичной ошибки \\(0.5*(Y-F1)^2\\), потому такой бустинг называют ["градиентный бустинг"](https://en.wikipedia.org/wiki/Gradient_boosting), а величины $(Y - F1)$ - "остатками" (residuals). 

Обычно для градиентного бустинга используют следующие библиотеки:

- XGBoost
- CatBoost
- LightGBM

В классе мы будем использовать реализацию в sklearn, а в домашней работе предлагаю использовать XGBoost или любую другую реализацию из этих трех.

## Задача

Обучить модель градиентного бустинга на датасете, провести подбор гиперпараметров (взять любые 3 штуки, их список можно получить через документацию к модели).

Сравнить качество с полученными ранее результатами.

In [19]:
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
# Ваш код здесь


## Стекинг

В случае, когда разные модели совершают разные ошибки, можно использовать стекинг.

In [40]:
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
estimators = [
    ('LogReg', LogisticRegression()),
    ('Tree', DecisionTreeClassifier())
]
clf=StackingClassifier(estimators=estimators, final_estimator=LogisticRegression())

## Задача 

Добавить в стекинг еще несколько классификаторов (можно взять классификаторы из прошлых заданий) и получить финальный результат.

In [ ]:
# Ваш код здесь


## Задача

Иногда нужно использовать модели, у которых интерфейс отличается от интерфейса моделей из sklearn и тогда приходится самостоятельно реализовывать стекинг.

Задача 

Алгоритм построения стекинга:
- Получить прогнозы базовых моделей
- Использовать прогнозы базовых моделей в качестве признаков для финализирующей модели
- Оформить модель в отдельную функцию

In [ ]:
# Ваш код здесь


## Задача

Построить стекинговую регрессионную модель для данных о стоимости домов. Подобрать гиперпараметры для каждой базовой модели, оценить финальное качество.

In [3]:
from sklearn.datasets import load_boston
data = load_boston()
 
# Ваш код здесь


# Домашняя работа

Будем работать с датасетом подержанных машин https://www.kaggle.com/adityadesai13/used-car-dataset-ford-and-mercedes.

Задача - построить прогноз цены машины по ее данным (год выпуска, пробег, расход, размер двигателя, тип коробки передач и тд). 

Это задача регрессии, поэтому нужно использовать модели регрессоры, а не классификаторы (например, не `StackingClassifier`, а `StackingRegressor`).

## Easy

Взять один файл с маркой машины.

Построить прогноз, используя модели линейной регрессии и случайного леса. Сравнить результаты, используя кросс-валидацию. Подобрать гиперпараметры моделей (для линейной регрессии можно использовать реализацию с градиентным спуском `SGDRegressor`). Оценить качество, используя метрику `r2_score`. Оценить важность признаков (параметр `feature_importance_` у случайного леса).

## Normal

Объединить в один датафрейм данные по всем маркам машин. Преобразовать категориальные признаки.

Построить еще несколько моделей, используя подбор гиперпараметров. Сравнить между собой все построенные модели.

## Hard

Построить модель стекинга над всеми моделями из задачи Normal + добавить один из градиентных бустингов XGBoost/LightGBM/CatBoost, если они не использовались. Удалось ли улучшить результат?

# Полезные ссылки

- [Наивный байесовский классификатор](http://datareview.info/article/6-prostyih-shagov-dlya-osvoeniya-naivnogo-bayesovskogo-algoritma-s-primerom-koda-na-python/)
- [AdaBoost](https://habr.com/ru/company/otus/blog/503888/)
- [K ближайших соседей (kNN)](https://habr.com/ru/post/149693/)
- [Метод опорных векторов (SVM)](https://habr.com/ru/post/105220/)